In [1]:
import numpy as np

In [2]:
class Particle:
    def __init__(self, num_courses, num_slots, timetable_data):
        self.timetable_data = timetable_data  # Assign timetable_data to the Particle object
        self.position = None  # Initialize position attribute
        self.initialize_position(num_courses, num_slots)  # Initialize position randomly
        self.velocity = np.random.rand(num_courses)  # Initialize velocity randomly
        self.best_position = self.position.copy()  # Initialize best position as current position
        self.fitness = None  # Initialize fitness value

    def initialize_position(self, num_courses, num_slots):
        # Initialize position randomly while satisfying constraints
        # Ensure each day of the week has only 4 courses
        num_days = 6  # Assuming 5 days in a week
        courses_per_day = num_courses // num_days  # Courses per day
        remaining_courses = num_courses % num_days  # Remaining courses to distribute

        position = np.zeros(num_courses, dtype=int)

        # Distribute courses evenly over the days of the week
        for day in range(num_days):
            start_index = day * courses_per_day + min(day, remaining_courses)
            end_index = start_index + courses_per_day + (1 if day < remaining_courses else 0)
            # Modify this part to assign time slots using PSO algorithm
            assigned_slots = self.assign_time_slots(end_index - start_index, num_slots)
            position[start_index:end_index] = assigned_slots

        self.position = position  # Set the position attribute

    def assign_time_slots(self, num_courses, num_slots):
        # This method assigns time slots to courses using PSO algorithm
        # Here, for simplicity, it randomly assigns time slots
        return np.random.choice(num_slots, num_courses, replace=False)


In [3]:
def convert_to_time_slots(timetable_data, numerical_timetable):
    time_slots = []
    for slot_index in numerical_timetable:
        time_slots.append(timetable_data[slot_index]['time_slot'])
    return time_slots

In [4]:
def update_velocity(particle, global_best_position, inertia_weight, cognitive_param, social_param):
    inertia_term = inertia_weight * particle.velocity
    cognitive_term = cognitive_param * np.random.rand() * (particle.best_position - particle.position)
    if global_best_position is not None:  # Check if global_best_position is not None
        social_term = social_param * np.random.rand() * (global_best_position - particle.position)
    else:
        social_term = np.zeros_like(inertia_term)  # If None, set social_term to zero vector
    particle.velocity = inertia_term + cognitive_term + social_term


In [5]:
def fitness_function(position, timetable_data):
    # Initialize fitness score
    fitness_score = 0
    
    # Check for conflicts in the timetable
    for course_index, slot_index in enumerate(position):
        if course_index >= len(timetable_data):  # Check if course_index is valid
            continue  # Skip if index is out of range
        
        # Get the room and time slot assigned to the current course
        room_assigned = timetable_data[course_index]['room']
        time_slot_assigned = timetable_data[course_index]['time_slot']
        
        # Check if there is a room conflict (two courses assigned to the same room at the same time)
        for other_course_index, other_slot_index in enumerate(position):
            if other_course_index != course_index and other_course_index < len(timetable_data):  # Check if other_course_index is valid
                if slot_index == other_slot_index:
                    other_room_assigned = timetable_data[other_course_index]['room']
                    if room_assigned == other_room_assigned:
                        fitness_score += 1  # Increment fitness score for room conflict
                    
        # Check if there is a teacher conflict (two courses taught by the same teacher at the same time)
        if slot_index is not None:
            teacher_assigned = timetable_data[course_index]['teacher']
            for other_course_index, other_slot_index in enumerate(position):
                if other_course_index != course_index and other_course_index < len(timetable_data):  # Check if other_course_index is valid
                    if slot_index == other_slot_index:
                        other_teacher_assigned = timetable_data[other_course_index]['teacher']
                        if teacher_assigned == other_teacher_assigned:
                            fitness_score += 1  # Increment fitness score for teacher conflict
                # Check if there is a conflict where no course is assigned at a specific time slot
        if slot_index is None:
            continue  # Skip if slot_index is None (no course assigned)
        else:
            # Check if there is any other course assigned to the same time slot
            if np.count_nonzero(position == slot_index) > 1:
                fitness_score += 1  # Increment fitness score for no course assigned conflict
        
    return fitness_score

In [6]:
def update_position(particle):
    particle.position = particle.position + particle.velocity

In [7]:
def pso(num_particles, num_courses, num_slots, max_iterations, timetable_data):
    particles = [Particle(num_courses, num_slots, timetable_data) for _ in range(num_particles)]
    global_best_position = None
    global_best_fitness = float('inf')

    inertia_weight = 0.5
    cognitive_param = 1.5
    social_param = 2.0

    for _ in range(max_iterations):
        for particle in particles:
            particle.fitness = fitness_function(particle.position, timetable_data)
            if particle.fitness < fitness_function(particle.best_position, timetable_data):
                particle.best_position = particle.position.copy()
                if particle.fitness < global_best_fitness:
                    global_best_position = particle.position.copy()
                    global_best_fitness = particle.fitness

        for particle in particles:
            update_velocity(particle, global_best_position, inertia_weight, cognitive_param, social_param)
            update_position(particle)

    # Round the positions to nearest integers for better interpretation
    if global_best_position is not None:
        global_best_position_rounded = np.round(global_best_position).astype(int)
        return global_best_position_rounded
    else:
        return None

In [8]:
def print_timetable(timetable_data, numerical_timetable):
    num_days = 6  
    # Organize timetable data by days and time slots
    timetable_by_day = [[] for _ in range(num_days)]
    for course_index, slot_index in enumerate(numerical_timetable):
        day_index = course_index // 5  # 3 courses per day
        if day_index < num_days:  # Check that today is within the allowed range.
            timetable_by_day[day_index].append((timetable_data[course_index], slot_index))

    # Print timetable for each day
    for day_index, day_courses in enumerate(timetable_by_day):
        print(f"Day {day_index + 1}:")
        for time_slot in range(num_slots):  # Assuming time slots per day
            print(f"Time Slot {time_slot}:")
            for course, slot_index in day_courses:
                if slot_index == time_slot:
                    print(f"  {course['course_name']} - Room: {course['room']} - Teacher: {course['teacher']}")
            print()  # Empty line between time slots
        print()  # Empty line between days


In [9]:
# Example usage
num_particles = 600
num_slots = 6
max_iterations = 250


# Number of courses
max_courses = 40  

num_courses = int(input("Enter the number of courses: "))
if num_courses > max_courses:
    print(f"Maximum number of courses allowed is {max_courses}. Please enter a valid number of courses.")
    num_courses = int(input("Enter the number of courses: "))

#num_courses = int(input("Enter the number of courses: "))

# Input course details from the user
timetable_data = []
for i in range(num_courses):
    course_name = input(f"Enter the name of course {i + 1}: ")
    room = input(f"Enter the room for course {course_name}: ")
    teacher = input(f"Enter the teacher for course {course_name}: ")
    # Remove time slot input from user
    timetable_data.append({'course_name': course_name, 'room': room, 'teacher': teacher, 'time_slot': None})

best_timetable = pso(num_particles, num_courses, num_slots, max_iterations, timetable_data)
if best_timetable is not None:
    print("Timetable of the week:")
    print_timetable(timetable_data, best_timetable)
else:               
    print("No valid timetable found.")

Enter the number of courses: 33
Enter the name of course 1: math2
Enter the room for course math2: 6a
Enter the teacher for course math2: a
Enter the name of course 2: elec
Enter the room for course elec: 6a
Enter the teacher for course elec: b
Enter the name of course 3: discrete
Enter the room for course discrete: 6a
Enter the teacher for course discrete: c
Enter the name of course 4: it
Enter the room for course it: 17b
Enter the teacher for course it: d
Enter the name of course 5: pl1
Enter the room for course pl1: 17b
Enter the teacher for course pl1: e
Enter the name of course 6: state2
Enter the room for course state2: 17b
Enter the teacher for course state2: f
Enter the name of course 7: math2
Enter the room for course math2: 6b
Enter the teacher for course math2: j
Enter the name of course 8: discrete
Enter the room for course discrete: 6b
Enter the teacher for course discrete: c
Enter the name of course 9: elec
Enter the room for course elec: 6b
Enter the teacher for course e

In [10]:
# GUI
from tkinter import *
from tkinter import ttk
from tkinter import messagebox

In [16]:
def solve_timetable(timetable_data):
    # PSO parameters
    num_particles = 600
    num_slots = 4
    max_iterations = 250
    num_courses = len(timetable_data)

    # Solve the timetable using PSO algorithm
    best_timetable = pso(num_particles, num_courses, num_slots, max_iterations, timetable_data)

    if best_timetable is not None:
        # Create and display the GUI window
        root = Tk()
        root.title("University Timetable")
        time_slots = {0: "8:00", 1: "10:00", 2: "12:00", 3: "14:00"}

        # Create a table widget
        days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
        table = ttk.Treeview(root, columns=days, show="")

        # Organize timetable data by days and time slots
        timetable_by_day = [[] for _ in range(5)]  # 4 time slots per day
        for course_index, slot_index in enumerate(best_timetable):
            if slot_index is not None:
                day_index = course_index // 5  # 4 courses per day
                timetable_by_day[day_index].append((timetable_data[course_index], time_slots[slot_index]))
        

        # Sort the timetable_by_day based on the time_slot
        for day_courses in timetable_by_day:
            day_courses.sort(key=lambda x: x[1], reverse=True)  # Sort courses by time_slot


        # Add rows for time slots and course assignments
        for day_index, day_courses in enumerate(timetable_by_day):
            table.insert("", "end", values=[days[day_index]])
            for course, time_slot in day_courses:
                table.insert("", "end", values=[f"{time_slot}: {course['course_name']} - {course['room']} - {course['teacher']}"])

        # Set the table to expand vertically as more rows are added
        table.pack(fill=BOTH, expand=True, padx=10, pady=10)

        # Run the GUI event loop
        root.mainloop()
    else:
        messagebox.showinfo("No valid timetable found.")



In [17]:
def generate_table():
    global tree
    columns = ('Teacher Name', 'Course Name', 'Room')
    tree = ttk.Treeview(main_window, columns=columns, show='headings')
    tree.heading('Teacher Name', text='Teacher Name')
    tree.heading('Course Name', text='Course Name')
    tree.heading('Room', text='Room')
    tree.pack()

In [18]:
def add_data():
    teacher_name = teacher_name_entry.get()
    course_name = course_name_entry.get()
    room = room_entry.get()
    timetable_data.append({'course_name': course_name, 'room': room, 'teacher': teacher_name, 'time_slot': None})
    tree.insert('',END, values=(teacher_name, course_name, room))
    teacher_name_entry.delete(0, END)
    course_name_entry.delete(0, END)
    room_entry.delete(0,END)

In [19]:

main_window = Tk()
main_window.geometry('1000x500')
main_window.title("University Timetable")

contact_information = []
timetable_data = []

generate_table()

label_1 = Label(main_window, text="Teacher Name:")
label_1.pack()
teacher_name_entry = Entry(main_window)
teacher_name_entry.pack()

label_2 = Label(main_window, text="Course Name:")
label_2.pack()
course_name_entry = Entry(main_window)
course_name_entry.pack()

label_3 = Label(main_window, text="Room:")
label_3.pack()
room_entry = Entry(main_window)
room_entry.pack()

def solve():
    solve_timetable(timetable_data)

def clear_table():
    tree.delete(*tree.get_children())

clear_button = Button(main_window, text="Clear Table", command=clear_table)
clear_button.place(x=650, y=400)

add_button = Button(main_window, text="Add", command=add_data)
add_button.place(x=550, y=400)

solve_button = Button(main_window, text="Solve", command=solve)
solve_button.place(x=400, y=400)

main_window.mainloop()